# Tutorial & Evaluation

In [1]:
import sys
import torch
import yaml
from tqdm import tqdm

from fromage.vqa_dataset import VQA_RADDataset
from fromage.model import Fromage, FromageModel
from fromage.experiment import Experiment
from fromage.data import MIMICDataset, cxr_image_transform
from fromage.utils import preprocess_report
from evaluate import load # if throws error, please run the following command "pip instal evaluate"

/kuacc/users/oince22/.conda/envs/fromage/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## config
ckpt_path = "../logs/checkpoints/lm_med_vis_med/last.ckpt"
config_path = "../config/train-untied_lm_med_vis_med.yaml"
dataset_path = "../data/datasets/VQA_RAD"

In [3]:
transform = cxr_image_transform(resize=512, center_crop_size=480, train=False) 
dataset = VQA_RADDataset(dataset_path, transform)

In [4]:
dataset[0] # returns image, question, answer

(tensor([[[0.0667, 0.0667, 0.0667,  ..., 0.1882, 0.1725, 0.1412],
          [0.0667, 0.0667, 0.0667,  ..., 0.2039, 0.1961, 0.2000],
          [0.0667, 0.0667, 0.0667,  ..., 0.2235, 0.2157, 0.2235],
          ...,
          [0.0784, 0.0706, 0.0706,  ..., 0.0824, 0.0706, 0.0902],
          [0.0784, 0.0784, 0.0784,  ..., 0.0824, 0.0706, 0.0902],
          [0.0784, 0.0784, 0.0784,  ..., 0.0863, 0.0784, 0.0902]],
 
         [[0.0667, 0.0667, 0.0667,  ..., 0.1882, 0.1725, 0.1412],
          [0.0667, 0.0667, 0.0667,  ..., 0.2039, 0.1961, 0.2000],
          [0.0667, 0.0667, 0.0667,  ..., 0.2235, 0.2157, 0.2235],
          ...,
          [0.0784, 0.0706, 0.0706,  ..., 0.0824, 0.0706, 0.0902],
          [0.0784, 0.0784, 0.0784,  ..., 0.0824, 0.0706, 0.0902],
          [0.0784, 0.0784, 0.0784,  ..., 0.0863, 0.0784, 0.0902]],
 
         [[0.0667, 0.0667, 0.0667,  ..., 0.1882, 0.1725, 0.1412],
          [0.0667, 0.0667, 0.0667,  ..., 0.2039, 0.1961, 0.2000],
          [0.0667, 0.0667, 0.0667,  ...,

In [5]:
with open(config_path) as file:
    config = yaml.safe_load(file)
    
model = Experiment(config)
model = model.load_from_checkpoint(ckpt_path)
model = model.model.to("cuda")
model.device = "cuda"

/kuacc/users/oince22/.conda/envs/fromage/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/kuacc/users/oince22/.conda/envs/fromage/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/kuacc/users/oince22/.conda/envs/fromage/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/kuacc/users/oince22/.conda/envs/fromage/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


In [6]:
img, question, answer = dataset[0]
prompt = str("Question: " + question + " Answer (yes or no): ")
print("Prompt: ", prompt)

model.eval()
classes = ["yes", "no"]
with torch.inference_mode():
    prompts = [img, prompt] 
    print(model.generate_for_images_and_texts(prompts, top_p=0.9, temperature=0.5))
    print("Model Answer: ", classes[model.classification_for_eval(prompts, classes)])
    
print("Correct Answer: ", answer)

Prompt:  Question: Are the lungs normal appearing? Answer (yes or no): 
 Chronic bronchitis. Aortic valve. Left ventricular failure. Right ventricular failure. Left ventricular failure. Left ventricular failure. Left vent
bsz: 1, seq_len: 17, dim: 768
logits.shape: torch.Size([1, 50267])
probs.shape: torch.Size([1, 50267])
cur_tok_prob.shape: torch.Size([1, 1])
curr_ppl: tensor([[8640.8291]], device='cuda:0')
min_ppl: tensor([[8640.8291]], device='cuda:0')
bsz: 1, seq_len: 17, dim: 768
logits.shape: torch.Size([1, 50267])
probs.shape: torch.Size([1, 50267])
cur_tok_prob.shape: torch.Size([1, 1])
curr_ppl: tensor([[293477.9062]], device='cuda:0')
min_ppl: tensor([[293477.9062]], device='cuda:0')
Model Answer:  yes
Correct Answer:  No


# Evaluation

In [14]:
transform = cxr_image_transform(resize=512, center_crop_size=480, train=False) 
dataset_closed = VQA_RADDataset(dataset_path, transform, 'closed')
dataset_open = VQA_RADDataset(dataset_path, transform, 'open')

print(dataset_closed.get_len())
print(dataset_open.get_len())

511
283


## Closed dataset: accuracy

In [ ]:
import string 

right_answers = 0
total_answers = 0

def get_model_response(prompts):
    model_ans_full = model.generate_for_images_and_texts(prompts, top_p=0.9, temperature=0.5)
    model_ans = model_ans_full.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
    try: 
        model_ans = model_ans.split()[0] # take only the first word, sometimes model makes a whole sentence
        return model_ans
    except:
        return model_ans

for idx in tqdm(dataset_closed):
    img, q, ans = idx 
    with torch.inference_mode():
        model.eval()
        prompts = [idx[0], str("Question: " + idx[1] + " Yes/No answer: ")] 
        for _ in range(4): # try 5 times to get the correct answer
            model_ans = get_model_response(prompts)
            if model_ans.lower() == ans.lower():
                right_answers += 1
                break
            else:
                pass
        total_answers += 1        

print(right_answers, '/', total_answers )
print((right_answers/total_answers)*100, '% correct')

## Open dataset: Bleu score

In [ ]:
exact_match_metric = load("bleu")

In [ ]:
# example
predictions=['how are you?']
references=['hello how are you?']
results = exact_match_metric.compute(predictions=predictions, references=references)
print(results)

In [ ]:
total_bleu_score = 0
total = 0

for idx in tqdm(dataset_open):
    img, q, ans = idx 
    with torch.inference_mode():
        model.eval()
        prompts = [idx[0], str("Question: " + idx[1] + " Answer: ")] 
        model_ans_full = model.generate_for_images_and_texts(prompts, top_p=0.9, temperature=0.5)    
        current_bleu_scores = []
        for _ in range(4): # try 5 times, get the best score of those 5 times
            try:
                bleu_score = exact_match_metric.compute(predictions=[model_ans_full], references=[ans]).get('bleu')
                current_bleu_scores.append(bleu_score)
            except:
                pass
        if len(current_bleu_scores) > 1:
            total_bleu_score += max(current_bleu_scores) # you can also take the average
        total += 1
        
print(total_bleu_score, '/', total)
print("bleu score: ", total_bleu_score/total)